# Database Creation & Management

Notebook to outline what data is stored, where it comes from and how it is created from the raw data

In [22]:
import datetime as dt
import numpy as np
import pandas as pd
import warnings

from epl.matches_parse import fetch_file, get_all_curr_urls, get_country_urls, get_most_recents, get_register, handle_initial_match_db
from epl.query import create_conn, create_and_query, get_root_dir, get_table_columns, query_db

pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

## 1. Raw Data Source

All the raw data (currently) for the database comes from the football-data website [here](https://www.football-data.co.uk/)

It consists of raw data for matches played across premier and lower divs in Europe, with data on:
 - Match date, teams playing
 - Half time and full time score line and full time result (A, D, H)
 - Shots, shots on target, fouls, yellows, reds
 - Many bookies closing pre-match odds for:
  - Fulltime result
  - Over/under 2.5

### 1a. Raw Data Types

__Matches__

The main raw data type is historical match data - this can be used to train models and is available at football-data's historical data section [here](https://www.football-data.co.uk/data.php)

Data is provided as csvs per season, per div, per country and so can be scraped by iterating as such

__Fixtures__

Data is provided as csvs in a similar format to match data [here](https://www.football-data.co.uk/matches.php)

## 2. Identifying & Fetching Raw Data

Once we have our data sources, the problem then becomees importing the data from the website to ensure:
 - Same column data stored for each match / fixture
 - Datatypes are correct for eahc column
 - We can run the same process regularly/daily and only update with the incremental new data
 
Let's look at getting the data into a suitable format for saving down

### 2a. Identifying csv Files on Webpage

The first task is to scrape (using BeautifulSoup) the webpage to identify:
 - Which countries we have data for
 - Per country, which divs and seasons we have data for
 - Per country, which data is the 'most recent' historical data and so needs refreshed (as may be new results stored here)

The below code scans the historical dtaa page and returns the country specific urls where we can find the raw dtaa for each country

In [6]:
cty_urls = get_country_urls()
cty_urls

{'england': 'https://www.football-data.co.uk/englandm.php',
 'scotland': 'https://www.football-data.co.uk/scotlandm.php',
 'germany': 'https://www.football-data.co.uk/germanym.php',
 'italy': 'https://www.football-data.co.uk/italym.php',
 'spain': 'https://www.football-data.co.uk/spainm.php',
 'france': 'https://www.football-data.co.uk/francem.php',
 'netherlands': 'https://www.football-data.co.uk/netherlandsm.php',
 'belgium': 'https://www.football-data.co.uk/belgiumm.php',
 'portugal': 'https://www.football-data.co.uk/portugalm.php',
 'turkey': 'https://www.football-data.co.uk/turkeym.php',
 'greece': 'https://www.football-data.co.uk/greecem.php'}

Once we have them, then we need to scan each of them to get the relevant links to the csv data

The below function returns a dataframe of all the links that exist across the coutnry pages in a ordered fashion

In [10]:
df_links = get_all_curr_urls(cty_urls)
df_links

,Date,Country,DivName,Div,Season,url
0,2020-12-15,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...
1,2020-12-15,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...
2,2020-12-15,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...
3,2020-12-15,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...
4,2020-12-15,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...
...,...,...,...,...,...,...
574,2020-12-15,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...
575,2020-12-15,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...
576,2020-12-15,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...
577,2020-12-15,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...


An example of the raw data below:

In [15]:
specific_url = df_links['url'][0]
print(specific_url)
fetch_file(specific_url).tail()

https://www.football-data.co.uk/mmz4281/2021/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E0.csv


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
112,E0,13/12/2020,12:00,Southampton,Sheffield United,3,0,H,1,0,H,A Madley,16,3,6,0,8,14,8,4,0,1,0,0,1.70,3.75,5.00,1.70,3.8,5.00,1.73,3.50,4.90,1.73,4.01,5.07,1.70,3.8,5.00,1.70,3.70,5.25,1.76,4.01,5.25,1.72,3.83,4.95,2.01,1.89,2.01,1.89,2.07,1.92,1.99,1.83,-0.75,1.97,1.96,1.97,1.94,1.98,1.97,1.94,1.93,1.61,3.80,5.25,1.65,3.90,5.25,1.63,3.95,5.25,1.65,4.11,5.75,1.63,3.90,5.50,1.60,4.0,5.75,1.68,4.25,5.81,1.64,4.03,5.45,1.80,2.0,1.86,2.05,1.90,2.15,1.82,2.01,-1.00,2.02,1.77,2.17,1.78,2.20,1.91,2.11,1.78
113,E0,13/12/2020,14:15,Crystal Palace,Tottenham,1,1,D,0,1,A,K Friend,16,14,5,6,15,11,4,7,1,1,0,0,4.50,3.90,1.72,4.75,3.7,1.75,4.60,3.50,1.77,4.92,3.83,1.79,4.75,3.7,1.75,4.80,3.70,1.75,4.92,3.94,1.82,4.71,3.75,1.76,1.90,1.90,1.96,1.94,1.97,1.99,1.90,1.91,0.75,1.90,2.03,1.90,2.03,1.94,2.03,1.88,1.98,4.00,3.80,1.85,4.10,3.80,1.83,4.00,3.70,1.85,4.33,3.93,1.85,4.20,3.70,1.83,4.50,3.7,1.80,4.60,3.93,1.91,4.21,3.77,1.85,1.80,2.0,1.86,2.05,1.91,2.11,1.84,1.97,0.50,2.07,1.86,2.08,1.85,2.11,1.88,2.03,1.84
114,E0,13/12/2020,16:30,Fulham,Liverpool,1,1,D,1,0,H,A Marriner,10,12,5,6,9,5,6,8,3,1,0,0,9.00,5.50,1.30,8.50,5.5,1.33,8.00,5.25,1.33,8.67,5.98,1.34,8.50,5.5,1.33,8.00,5.25,1.36,9.30,6.00,1.37,8.44,5.66,1.34,1.40,3.00,1.44,2.95,1.44,3.05,1.42,2.84,1.50,1.99,1.94,1.99,1.93,2.01,1.96,1.96,1.90,8.00,5.50,1.33,7.75,5.50,1.35,7.50,5.50,1.35,8.51,5.36,1.38,8.50,5.25,1.35,8.50,5.0,1.36,9.30,5.80,1.40,8.24,5.43,1.35,1.53,2.5,1.52,2.66,1.53,2.82,1.49,2.63,1.50,1.90,2.03,1.91,2.01,1.97,2.06,1.89,1.97
115,E0,13/12/2020,19:15,Arsenal,Burnley,0,1,A,0,0,D,G Scott,18,10,6,2,5,11,13,4,2,1,1,0,1.53,4.20,6.25,1.55,4.1,6.25,1.55,3.95,6.50,1.54,4.33,6.84,1.53,4.2,6.50,1.53,4.10,6.50,1.58,4.48,7.00,1.54,4.22,6.42,1.90,1.90,1.95,1.95,1.97,1.98,1.90,1.92,-1.00,1.97,1.96,1.97,1.95,2.00,2.01,1.92,1.94,1.40,4.75,8.00,1.42,4.75,7.25,1.47,4.40,6.75,1.49,4.82,6.88,1.42,4.50,8.00,1.44,4.5,7.50,1.52,4.83,8.65,1.46,4.62,7.17,1.72,2.1,1.74,2.21,1.79,2.26,1.72,2.13,-1.25,2.07,1.86,2.09,1.85,2.13,1.92,2.04,1.84
116,E0,13/12/2020,19:15,Leicester,Brighton,3,0,H,3,0,H,M Atkinson,15,12,6,4,9,11,9,5,2,1,0,0,2.00,3.50,3.80,2.00,3.4,3.90,2.00,3.45,3.75,2.06,3.56,3.91,2.00,3.5,3.75,2.00,3.50,3.80,2.11,3.72,3.93,2.03,3.49,3.76,1.90,1.90,1.94,1.96,1.99,1.98,1.91,1.91,-0.50,2.06,1.87,2.06,1.88,2.08,1.89,2.02,1.84,2.00,3.50,3.75,1.95,3.60,3.90,2.05,3.50,3.55,2.12,3.55,3.72,2.00,3.50,3.80,2.05,3.5,3.60,2.13,3.70,4.07,2.04,3.52,3.70,1.80,2.0,1.88,2.03,1.92,2.15,1.83,1.99,-0.50,2.10,1.83,2.12,1.83,2.13,1.88,2.05,1.82


### 2b. Processing Raw Data

For each url, the data is:
 - Fetched (using requests)
 - Loaded into a pd.DataFrame

Once we have a list of dataframes then we can concat them altogether (outer join)

Then we can:
 - Convert any date columns to a consist datetime format
 - Remove any columns that have >99% nulls (potentially bad raw data or poor parsing)

## 3. Storing Data

Data is stored using sqlite given it is:
 - Easiy integrated with pandas
 - Suitable given the datasize is not large (I find sqlite v slow and inflexible vs e.g. kdb+)
 - Quick and easy to implement and understand
 
This manifests as a simple single file_name.sqlite file where all the data is stored as a relational db i.e. every table is a list of (row_id, row_data) pairs

The below function handles this for the match data (and the fixture data is very similar)

In [17]:
?? handle_initial_match_db

Signature:  handle_initial_match_db(df_new, uat=False)
Source:   
def handle_initial_match_db(df_new, uat=False):
    '''
    Function to handle initial creation of matches table in sqlite
    Handled separately in order to determine which columns to keep
    All updates then will be joins where columns must be strict subset of existing db
    i.e. can't have new columns created  - seems okay restriction as data feed relatively mature
    '''
    # concat all the res dfs together
    dfs = df_new[df_new.Status == 'Processed']
    df = pd.concat(list(dfs['Results']))
    # remove col if >99% of col are NaNs in combined df
    df = df[df.columns[df.isnull().mean() < 0.99]]
    df = clean_data(df)

    # set down data into db
    try:
        conn = create_conn(uat=uat)
        print('Connection established - setting down intial db')
        df.to_sql('matches', conn, index=False)
        conn.close()
        df_new['Status'] = 'Processed'
        df_new['ParseMessage'] = ''
    except:
 

## 4. Daily Updating / Management

The process of identifying, fetching, cleaning/processing the raw data is fairly straight forward

The issue then becomes how to ensure all data remains up to date in a way that identifies parsing errors and doesn't revolve around a daily process that relies on previous daily process runs e.g. everything breaks because one day once it didn't work

This is managed by the below parsing infra

### 4a. Registers

For tables where raw data is parsed i.e. requires some kind of data fetching from an external source (compared to feature table creation where the input data is the raw matches data we have stored), we need to keep track of what we have done

For this, we maintain a csv file called a 'register'

Each day (or regular time interval - as this is a more data-streaming like setup works for longer or shorter intervals), we wish to do the following:
 - Query raw data source to see what is listed there (csv url links)
 - Identify what we already have stored in the database
 - If the table already exists, just append on any potentially new data to keep us up to date
 - If not, then query everything possible, process and create the table in the database
 - Record what has been done (error logging etc)
 
The final step is crucial to e.g. being able to run the function many tims a day and prevent __and this is where registers come in__:
 - Repeatedly getting new data even though this process has already been done for the day
 - Being able to re-run iff (and only) there were errors


### 4b. Example Process

__Most Recent Season__

For match data, we want to know what the most recent seasons are per country on each attempt to run the update so we only load potentially new data

This looks like the below:

In [21]:
# go to the data source, parse page, get most recent url
get_most_recents(cty_urls)

,Date,Country,MostRecentSeason
0,2020-12-15,england,2021
1,2020-12-15,scotland,2021
2,2020-12-15,germany,2021
3,2020-12-15,italy,2021
4,2020-12-15,spain,2021
5,2020-12-15,france,2021
6,2020-12-15,netherlands,2021
7,2020-12-15,belgium,2021
8,2020-12-15,portugal,2021
9,2020-12-15,turkey,2021


We can then save this data daily as a csv register for use in the parsing process

In [23]:
get_register('register_most_recents')

,Date,Country,MostRecentSeason
0,2020-12-03,england,2021
1,2020-12-03,scotland,2021
2,2020-12-03,germany,2021
3,2020-12-03,italy,2021
4,2020-12-03,spain,2021
...,...,...,...
83,2020-12-15,netherlands,2021
84,2020-12-15,belgium,2021
85,2020-12-15,portugal,2021
86,2020-12-15,turkey,2021


__Match Register__

Now we need to:
 - Use the above most recent season info
 - Use knowledge of what already has been parsed into the db
to understand what raw data files are new and so we should pick up and parse

For this we use/maintain a __match_register__

In [24]:
get_register('register_matches')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,
1,2020-12-03,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,
2,2020-12-03,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,
3,2020-12-03,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,
4,2020-12-03,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,
...,...,...,...,...,...,...,...,...,...
728,2020-12-15,netherlands,Eredivisie,N1,2021,https://www.football-data.co.uk/mmz4281/2021/N...,True,Processed,
729,2020-12-15,belgium,Jupiler League,B1,2021,https://www.football-data.co.uk/mmz4281/2021/B...,True,Processed,
730,2020-12-15,portugal,Liga I,P1,2021,https://www.football-data.co.uk/mmz4281/2021/P...,True,Processed,
731,2020-12-15,turkey,Futbol Ligi 1,T1,2021,https://www.football-data.co.uk/mmz4281/2021/T...,True,Processed,


For this, we:
 - Pick up all the csvs from the webpage per country (as per 'get_all_curr_urls' above)
 - Determine whether they have previously been processed using 'Status' as 'Processed'
 - List a batch of new matches csvs (if any) along with the most recent match csvs per county
 
This allows us daily to only process completely new data, or the most recent season data that may contain new match outcomes

## 5. Feature Table Management

The above process is used to fetch, maintain and update raw match and fixture data from a webpage

For creating/training predictive models for scorelines/results, we will need to transform this raw data into 'features' (a set of X variables) to feed to statistical models

We can do this 'on-the-fly', but if we have a set of known variables then:
 - more efficient to calc and store them once
 - storing once speeds up model iteration as don't need to constantly wait for feature calculation
 - will result in end front-end response times being satisfactory

### 5a. Features and Fixtures_features

A features table is maintained both for:
 - historical match data: this is useful for training models
 - fixture data: this is used to make predictions using trained models on upcoming games (as we don't yet know the predicted quantity)

To create these, a dictionary object is maintained and post each raw data update (as per above) we:
 - check if there any new matches
 - if new matches, compute the features for those matches

## 6. Overview

The result of the above is a sqlite database that remains up to date with new fixtures and match results so we can:
 - continue to train models on the most recent data
 - continue to make predictions for the most recent unknown result fixtures

The database thus looks like this - a single file_name.sqlite with the following tables:
 - matches: The historical match data with basic id data, result, scoreline, stats, betting odds
 - features: The corresponding feature data for those matches
 - fixtures: The upcoming fixture data with basic id data, betting odds but no outcomes (scores, shots etc) as that hasn't happened yet
 - fixtures_features: The corresponding feature data for the upcoming fixtures

__Example File Structure__

In [38]:
'''
root/
    data/
        footie.sqlite
        register_fixtures.csv
        register_macthes.csv
        register_most_recents.csv
    scripts/
        parse.py (this is the script that runs/maintains/creates everything in 'data/'
''';

__Demo__

In [27]:
query_db('SELECT * FROM sqlite_master')

Running query: SELECT * FROM sqlite_master


,type,name,tbl_name,rootpage,sql
0,table,matches,matches,2,"CREATE TABLE ""matches"" (\n""Country"" TEXT,\n ""..."
1,table,fixtures,fixtures,3,"CREATE TABLE ""fixtures"" (\n""AsOfDate"" TIMESTAM..."
2,table,features,features,23598,"CREATE TABLE ""features"" (\n""Date"" TIMESTAMP,\n..."
3,table,fixtures_features,fixtures_features,102133,"CREATE TABLE ""fixtures_features"" (\n""Date"" TIM..."


We can query a table using the following that simplifies sql query statement creation for cols and whereclauses:

In [29]:
df_matches = create_and_query('matches',
                              cols=['Date', 'Country', 'Div', 'Season', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HS', 'AS', 'B365A', 'B365D', 'B365H'],
                              wc={'Season': ['IN', ['1920', '2021']]}
                             )

Running query: SELECT Date, Country, Div, Season, HomeTeam, AwayTeam, FTHG, FTAG, HS, [AS], B365A, B365D, B365H FROM matches WHERE Season IN ('1920', '2021')


In [31]:
df_matches.tail(10)

,Date,Country,Div,Season,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,B365A,B365D,B365H
8714,2020-12-13,spain,SP2,2021,Sp Gijon,Zaragoza,1.0,0.0,10.0,6.0,4.75,3.1,1.85
9447,2020-12-13,turkey,T1,2021,Erzurum BB,Genclerbirligi,0.0,1.0,14.0,11.0,3.40,3.3,2.10
9448,2020-12-13,turkey,T1,2021,Alanyaspor,Besiktas,2.0,1.0,20.0,10.0,2.90,3.5,2.30
9449,2020-12-13,turkey,T1,2021,Buyuksehyr,Gaziantep,1.0,2.0,19.0,9.0,6.00,4.0,1.53
9000,2020-12-14,france,F2,2021,Le Havre,Clermont,0.0,0.0,10.0,7.0,2.15,3.1,3.60
9529,2020-12-14,greece,G1,2021,Asteras Tripolis,Atromitos,2.0,0.0,17.0,2.0,3.60,3.0,2.20
8394,2020-12-14,italy,I2,2021,Reggina,Venezia,1.0,2.0,11.0,14.0,2.60,3.0,2.87
8517,2020-12-14,spain,SP1,2021,Celta,Cadiz,4.0,0.0,8.0,12.0,5.25,3.6,1.72
8715,2020-12-14,spain,SP2,2021,Girona,Vallecano,0.0,0.0,8.0,20.0,2.70,3.1,2.62
9450,2020-12-14,turkey,T1,2021,Sivasspor,Antalyaspor,0.0,0.0,9.0,12.0,4.00,3.5,1.90


In [32]:
# select all cols from feats
df_feats = create_and_query('features',
                            wc={'Season': ['IN', ['1920', '2021']]}
                           )

Running query: SELECT * FROM features WHERE Season IN ('1920', '2021')


In [33]:
df_feats.tail()

,Date,Team,Location,Country,Div,Season,AvgGF_3,AvgGA_3,AvgGFH_3,AvgGAH_3,AvgGFA_3,AvgGAA_3,AvgSF_3,AvgSA_3,AvgSFH_3,AvgSAH_3,AvgSFA_3,AvgSAA_3,AvgSTF_3,AvgSTA_3,AvgSTFH_3,AvgSTAH_3,AvgSTFA_3,AvgSTAA_3,AvgPPG_3,AvgPPGH_3,AvgPPGA_3,AvgGF_5,AvgGA_5,AvgGFH_5,AvgGAH_5,AvgGFA_5,AvgGAA_5,AvgSF_5,AvgSA_5,AvgSFH_5,AvgSAH_5,AvgSFA_5,AvgSAA_5,AvgSTF_5,AvgSTA_5,AvgSTFH_5,AvgSTAH_5,AvgSTFA_5,AvgSTAA_5,AvgPPG_5,AvgPPGH_5,AvgPPGA_5,AvgGF_10,AvgGA_10,AvgGFH_10,AvgGAH_10,AvgGFA_10,AvgGAA_10,AvgSF_10,AvgSA_10,AvgSFH_10,AvgSAH_10,AvgSFA_10,AvgSAA_10,AvgSTF_10,AvgSTA_10,AvgSTFH_10,AvgSTAH_10,AvgSTFA_10,AvgSTAA_10,AvgPPG_10,AvgPPGH_10,AvgPPGA_10,AvgGF_20,AvgGA_20,AvgGFH_20,AvgGAH_20,AvgGFA_20,AvgGAA_20,AvgSF_20,AvgSA_20,AvgSFH_20,AvgSAH_20,AvgSFA_20,AvgSAA_20,AvgSTF_20,AvgSTA_20,AvgSTFH_20,AvgSTAH_20,AvgSTFA_20,AvgSTAA_20,AvgPPG_20,AvgPPGH_20,AvgPPGA_20,AvgGF_40,AvgGA_40,AvgGFH_40,AvgGAH_40,AvgGFA_40,AvgGAA_40,AvgSF_40,AvgSA_40,AvgSFH_40,AvgSAH_40,AvgSFA_40,AvgSAA_40,AvgSTF_40,AvgSTA_40,AvgSTFH_40,AvgSTAH_40,AvgSTFA_40,AvgSTAA_40,AvgPPG_40,AvgPPGH_40,AvgPPGA_40,AvgGF_80,AvgGA_80,AvgGFH_80,AvgGAH_80,AvgGFA_80,AvgGAA_80,AvgSF_80,AvgSA_80,AvgSFH_80,AvgSAH_80,AvgSFA_80,AvgSAA_80,AvgSTF_80,AvgSTA_80,AvgSTFH_80,AvgSTAH_80,AvgSTFA_80,AvgSTAA_80,AvgPPG_80,AvgPPGH_80,AvgPPGA_80
19052,2020-12-14,Celta,Home,spain,SP1,2021,2.333333,1.666667,1.333333,2.333333,1.666667,1.666667,11.000000,11.333333,13.666667,9.000000,10.000000,11.000000,6.333333,4.000000,5.000000,4.333333,5.666667,3.666667,2.000000,1.000000,1.333333,1.8,2.0,1.2,2.2,1.2,1.6,11.2,10.2,14.8,8.4,9.8,11.4,5.4,4.0,4.8,3.8,4.6,3.8,1.4,1.2,1.0,1.1,1.9,1.8,1.8,0.9,1.6,10.5,10.0,12.9,9.2,9.3,10.7,4.2,3.9,5.1,3.2,4.1,3.7,0.9,1.2,0.8,1.10,1.70,1.25,1.25,0.95,1.35,10.55,10.40,12.30,9.20,8.60,12.40,4.15,3.65,4.40,3.05,3.80,4.05,1.00,1.25,0.95,1.125,1.425,1.325,1.300,0.875,1.625,10.500,10.950,12.200,10.400,8.825,13.025,4.275,3.750,4.425,3.625,3.400,4.600,1.025,1.325,0.700,1.125,1.4875,1.5375,1.3750,1.1125,1.8375,10.5875,11.7375,12.30,10.8000,9.6625,13.325,3.9750,4.1375,4.6125,3.8625,3.5375,4.8625,1.0125,1.4875,0.7750
19054,2020-12-14,Girona,Home,spain,SP2,2021,1.666667,0.000000,1.000000,0.333333,1.333333,0.666667,10.333333,12.333333,8.666667,10.666667,8.333333,9.666667,3.333333,3.000000,2.000000,2.333333,2.666667,3.666667,3.000000,2.000000,2.000000,1.0,0.4,0.8,0.6,1.2,0.8,8.4,10.2,9.6,10.4,7.6,15.0,2.4,2.8,3.2,2.6,2.6,4.4,1.8,1.4,1.6,1.0,0.7,1.0,0.6,0.8,1.1,8.6,12.7,9.5,9.3,9.5,12.3,2.9,3.5,2.8,3.0,3.2,3.9,1.5,1.9,1.4,0.95,0.95,1.20,0.65,0.70,1.05,9.70,10.75,11.40,8.70,10.45,10.50,3.15,3.35,3.65,2.65,3.75,3.55,1.55,2.00,1.10,0.875,0.875,1.100,0.850,0.925,1.375,10.975,9.575,12.350,9.150,10.150,12.025,3.725,3.100,4.150,3.050,3.425,4.200,1.475,1.650,0.975,1.025,1.1250,1.2875,1.0125,1.0875,1.4500,11.2125,10.7625,NaN,NaN,NaN,NaN,3.7625,3.7000,NaN,NaN,NaN,NaN,1.3125,1.6000,1.1625
19058,2020-12-14,Vallecano,Away,spain,SP2,2021,0.666667,0.666667,1.333333,0.666667,1.000000,1.000000,11.666667,8.000000,10.666667,8.000000,10.000000,9.000000,3.000000,1.333333,2.666667,1.666667,3.333333,2.000000,1.333333,2.333333,1.333333,1.2,0.8,1.2,0.6,0.8,1.2,9.8,8.0,11.8,8.8,11.2,8.4,3.4,1.8,3.0,2.4,4.0,2.4,2.0,2.0,0.8,1.0,0.9,1.8,0.8,0.7,1.1,11.5,8.6,12.3,7.8,11.3,7.0,3.5,2.4,4.0,2.4,3.9,2.4,1.4,2.3,1.1,1.20,1.00,1.45,0.70,1.10,1.30,11.85,7.45,12.60,7.95,11.10,8.95,3.85,2.45,4.30,2.60,4.05,3.00,1.55,2.10,1.05,1.275,1.000,1.400,0.950,1.175,1.625,11.850,8.450,13.550,8.375,11.850,10.625,4.175,2.800,4.700,2.975,3.850,3.900,1.575,1.750,0.900,1.275,1.2750,1.5250,1.1000,1.0750,1.4125,12.5875,9.5000,14.75,11.5375,12.7625,13.875,4.2375,3.4125,5.1625,4.3250,4.1750,5.6375,1.3250,1.7375,1.0750
19048,2020-12-14,Antalyaspor,Away,turkey,T1,2021,1.000000,1.333333,0.666667,1.333333,1.666667,3.000000,9.333333,9.333333,7.333333,15.000000,9.333333,19.000000,4.666667,4.000000,3.333333,6.000000,4.000000,5.666667,1.333333,1.000000,0.666667,1.2,1.6,0.8,1.0,1.2,2.2,8.8,14.4,9.0,13.0,8.6,16.6,4.0,5.2,3.6,5.0,3.4,4.8,1.0,1.4,0.6,1.0,1.6,1.2,1.0,1.3,1.6,8.8,14.8,